In [ ]:
#Add your name here
#Name: Ilyas Khaja

In [ ]:
import re
import pandas as pd
import math
import numpy as np

In [ ]:
#For this programs we are going to evaluate the ID3 on three different datasets:
#Change the path based on your directory address 
path="/content/gdrive/MyDrive/CS412/hw1/"


#for agaricus we don't have a separate validation set, so we are going to reuse the training set.
agaricus = ["agaricus/agaricuslepiotatrain1.csv",
              "agaricus/agaricuslepiotatrain1.csv",
              "agaricus/agaricuslepiotatest1.csv"]

dataset1 = ["data_sets1/training_set.csv",
            "data_sets1/validation_set.csv",
            "data_sets1/test_set.csv"]



In [ ]:
# Load data from a file. It returns a list of data points as well as the list of variable names
def read_data(filename):
    f = open(filename, 'r')
    p = re.compile(',')
    data = []
    header = f.readline().strip()
    varnames = p.split(header)
    namehash = {}
    for l in f:
        data.append([int(x) for x in p.split(l.strip())])
    return (data, varnames)

In [ ]:
dataset = agaricus
train_data, varnames = read_data(path+dataset[0])
#the last element in the list is the class value."

In [ ]:
#Here we have the neccessary data structure and auxilary functions for construcing the decision trees
#You don't need to modify the codes in this cell
class Node:
    """ Node class for a decision tree. """
    #var is the variable name for the node
    def __init__(self, var):
        self.var = var

    def classify(x):
        """ Handled by the subclasses. """
        return None

    def dump(self, indent):
        """ Handled by the subclasses. """
        return None


class Leaf(Node):
    #value is the label of the leaf node
    def __init__(self, value):
        Node.__init__(self, None);
        self.value = value

    def classify(self, x):
        return self.value

    def dump(self, indent):
        print(' %d' % self.value)


class Split(Node):
    #var: the variable that we create the split on
    #left and right are the branches for each side which are Nodes
    def __init__(self, var, left, right):
        Node.__init__(self, var)
        self.left = left
        self.right = right

    def classify(self, x):
        if x[self.var] == 0:
            return self.left.classify(x)
        else:
            return self.right.classify(x)
    
    #use to print out the tree recursively
    def dump(self, indent):
        if indent > 0:
            print('')
        for i in range(0, indent):
            print('| ', end='')
        print('%s = 0 :' % self.var,end='')
        self.left.dump(indent+1)
        for i in range(0, indent):
            print('| ', end='')
        print('%s = 1 :' % self.var,end='')
        self.right.dump(indent+1)


Helper function computes entropy of Bernoulli distribution with parameter p

In [ ]:
def entropy(p):
  # >>>> YOUR CODE GOES HERE <<<<
  if (p == 1 or p == 0):
     return 0

  return -1 * ((p * np.log2(p)) + ((1-p) * np.log2(1-p)))
  # For now, always return "0":

entropy(6/7)

Compute information gain for a particular split, given the counts 

ny_nxi : number of occurences of y=1 with x_i=1 for all i=1 to n (#y=1 $\wedge$ #xi =1)
(n is number of variables and each variable is binary)

nxi : number of occurrences of x_i=1 (#x_i)

ny : number of ocurrences of y=1

total: total number instances in this branch


In [ ]:
def infogain(ny_nxi, nxi, ny, total): #x_i = attribute, y = classification
    # >>>> YOUR CODE GOES HERE <<<<

    
    entropy_s = entropy(ny / total) #overall entropy

    entropy_sv1 = 0 if ( nxi == 0) else entropy(ny_nxi / nxi) #entropy for x_i = 1

    nxi_0 = total - nxi #number of occurences of x_i = 0

    ny_nxi_0 = ny - ny_nxi #number of occurences of y = 1 with x_i = 0

    entropy_sv0 = 0 if (nxi_0 == 0) else entropy(ny_nxi_0 / nxi_0)

    return entropy_s - (((nxi / total) * (entropy_sv1)) + ((nxi_0 / total) * (entropy_sv0)))
    # For now, always return "0":

#infogain(6, 8, 9, 14)

In [ ]:
#split_df: the portion of the dataframe that routed to the current branch


def select_attr(split_df): #calculate infogain by iterating thorugh remaining attributes, select max 
  #>>> YOUR CODE GOES HERE <<< 
  #get infogan from first attribute
  max_attr = split_df.columns[0]
  classLabel = split_df.columns[-1]

  ny_nxi = ((split_df[max_attr] == 1) & (split_df[classLabel] == 1)).sum()
  nxi = (split_df[max_attr] == 1).sum()
  ny = (split_df[classLabel] == 1).sum()
  total = split_df.shape[0]

  max_gain = infogain(ny_nxi, nxi, ny, total)

  cols = split_df.shape[1]
  for i in range(1, cols-1): #iterate until poisonous col
    attr = split_df.columns[i]

    ny_nxi = ((split_df[attr] == 1) & (split_df[classLabel] == 1)).sum()
    nxi = (split_df[attr] == 1).sum()
    ny = (split_df[classLabel] == 1).sum()
    total = split_df.shape[0]

    gain = infogain(ny_nxi, nxi, ny, total)
    #print("attribute =", attr, "calculated gain = ", gain)

    if (gain > max_gain):
      max_attr = attr
      max_gain = gain 

  return (max_attr, max_gain)

Build tree in a top-down manner, selecting splits until we hit a pure leaf or all splits look bad.


In [ ]:
#Recursive function for building the tree. Note that the vanilla ID3 stops when the nodes are pure (base condition for the recursion);
#config is potential dictionary of hyperparams that you can tune over validation dataset
#For potential list of hyperparams check here:
#https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier

def helper(data, prev_attr):
  (selected_attr, gain) = select_attr(data)
  classLabel = data.columns[-1]

 # print("in helper function, prev_attr= ", prev_attr, "selected _attr =", selected_attr, " and gain= ", gain)

  if gain == 0: #base case, there is no gain to be made by splitting further, stop recursion
    if (data[classLabel]).sum() > 0: #all examples are positive
      #print("returning poisonous leaf: attribute =", prev_attr, " total samples: ", (data['poisonous']).sum())
      return Leaf(1)
    else:
      #print("returning non-poisonous leaf: attribute =", prev_attr, " total samples: ", data.shape[0] - (data['poisonous']).sum())
      return Leaf(0)

  #partition data according to attribute selected
  d1 = data[(data[selected_attr] == 1)]
  d2 = data[(data[selected_attr] == 0)]

  d1.drop(columns=selected_attr)
  d2.drop(columns=selected_attr)

  n1 = helper(d1, selected_attr)
  n2 = helper(d2, selected_attr)

  if (n1.classify == 0): #left branch
    return Split(selected_attr, n1, n2)
  else:
    return Split(selected_attr, n2, n1)


  #node = Split(selected_attr, helper(d1, selected_attr), helper(d2, selected_attr))
  #return node

def build_tree(data, config = {}):
    # >>>> YOUR CODE GOES HERE <<<<
    # For now, always return a leaf predicting "1":
    #select first attribute
    (selected_attr, gain) = select_attr(data)
    #print(selected_attr, gain)

    #partition data according to attribute selected
    d1 = data[(data[selected_attr] == 1)]
    d2 = data[(data[selected_attr] == 0)]

    d1.drop(columns=selected_attr)
    d2.drop(columns=selected_attr)

    n1 = helper(d1, selected_attr)
    n2 = helper(d2, selected_attr)

    if (n1.classify == 0): #left branch
      return Split(selected_attr, n1, n2)
    else:
      return Split(selected_attr, n2, n1)

   # root = Split(selected_attr, helper(d1, selected_attr), helper(d2, selected_attr))
    
    #return root


Build the decision tree

In [ ]:
model = build_tree(data_df)

In [ ]:
model.dump(0)

Calcuating the accuracy

In [ ]:
#Assume the last column in the class label and returns one dataframe for labels and one for attributes
def separate_attributes_label(data):
  class_column = data.columns[-1]
  xdata = data.drop(columns=class_column);
  ydata = data[class_column];
  return xdata, ydata

In [ ]:
xtrain, ytrain = separate_attributes_label(data_df);

In [ ]:
def accuracy(model, xdata, ydata):
  correct = 0.0;
  for i in range(xdata.shape[0]):
    if model.classify(xdata.loc[i]) == ydata.loc[i]:
     # print("correctly classified ", xdata.loc[i], " = ", ydata.loc[i])
      correct += 1;
  return correct / xdata.shape[0]


In [ ]:
print("Train Accuracy: {}".format(accuracy(model, xtrain, ytrain)))

Now that we have all pieces working we can automate the training for any given dataset

In [ ]:
def evaluate(dataset, config={}):
  train = pd.read_csv(path+dataset[0])
  val = pd.read_csv(path+dataset[1]) 
  test = pd.read_csv(path+dataset[2]) 

  xtrain, ytrain = separate_attributes_label(train);
  xval, yval = separate_attributes_label(val);
  xtest, ytest = separate_attributes_label(test);

  model = build_tree(train, config)
  model.dump(0)

  print("Train Accuracy: {}".format(accuracy(model, xtrain, ytrain)))
  print("Val Accuracy: {}".format(accuracy(model, xval, yval)))
  print("Test Accuracy: {}".format(accuracy(model, xtest, ytest)))

In [ ]:
evaluate(agaricus)

In [ ]:
config={'min_example': 2}
evaluate(dataset1, config)